In [8]:
import numpy as np
import pandas as pd
from time import time
from matplotlib import pyplot as plt
from functools import partial
# ML Scikit Imports
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img


# a décommenter si problème avec le GPU de votre machine
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
#for gpu in physical_devices:
#    tf.config.experimental.set_memory_growth(gpu, True)


 # imports locaux
#from usefulCmdsAndFcns import load_batch, importData2, prepare_datasets, normalize_features, encode_labels, learningCurves, ConfusionEvaluation, display_conv_filters_kv


# links with google drive and colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
   from google.colab import drive
   drive.mount('/content/gdrive')
   code_folder = '/content/gdrive/My Drive/machine_learning/Reinforcement Learning' # remplacer par le chemin vers ce notebook à partir de My Drive/...
   %cd "$code_folder"
   !ls "$code_folder"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/machine_learning/Reinforcement Learning
'Copie de sound.ipynb'	 results		 sound_memory.ipynb
 features_prof.csv	 sound_brouillon.ipynb	 sound_model_raspberry.ipynb
 features_test.csv	 sound.ipynb
 features_train.csv	 sound_lite.ipynb


In [9]:
# LABEL_PATH = r'./features_prof.csv'
# data = pd.read_csv(LABEL_PATH)
# data.head()

# feature_values = data.iloc[:, :512]
# label_names = data.iloc[:, 512]

# X_train, X_test, y_train, y_test = train_test_split(feature_values,
#                                                                                             label_names,
#                                                                                             test_size=0.5,
#                                                                                           stratify=np.array(
#                                                                                                 label_names),
#                                                                                             random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_test,
#                                                                                             y_test,
#                                                                                             test_size=0.5,
#                                                                                             stratify=np.array(
#                                                                                                 y_test),
#                                                                                            random_state=42)  


LABEL_PATH_TRAIN = r'./features_train.csv'
LABEL_PATH_TEST = r'./features_test.csv'
LABEL_PATH = r'./features_prof.csv'

data_train = pd.read_csv(LABEL_PATH_TRAIN)
data_train.head()

data_test = pd.read_csv(LABEL_PATH_TEST)
data_test.head()

data = pd.read_csv(LABEL_PATH_TEST)
data.head()

X_train = data_train.iloc[:, :512]
y_train = data_train.iloc[:, 512]

X_test = data_test.iloc[:, :512]
y_test = data_test.iloc[:, 512]

feature_values = data.iloc[:, :512]
label_names = data.iloc[:, 512]
m = y_train.shape[0]
print(m)
n = y_test.shape[0]
print(n)


labelNames_unique = label_names.unique()

label_nb = labelNames_unique.shape[0]

# enc labelNames to indices
encName2Ind = preprocessing.LabelEncoder()
encName2Ind.fit(labelNames_unique)
labelIndices_unique = encName2Ind.transform(labelNames_unique)
labelIndices  = encName2Ind.transform(label_names)

y_train = encName2Ind.transform(y_train)
y_test = encName2Ind.transform(y_test)

X_train = np.array(X_train)
X_test = np.array(X_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = y_train.reshape((m,1))
y_test = y_test.reshape((n,1))

# print(X_train.shape, y_train.shape)



670
330


In [10]:
RESULT_PATH = './results/'

predict=[]
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(RESULT_PATH+'model_from_keras_model.tflite')
interpreter.allocate_tensors() 


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(X_test, dtype="float32")

for i in range(0,n):
  interpreter.set_tensor(input_details[0]['index'], input_data[i].reshape((1,512)))
  interpreter.invoke()  output_data = interpreter.get_tensor(output_details[0]['index'])

  predict.append(np.argmax(output_data))



[1, 6, 0, 7, 0, 1, 0, 8, 4, 1]


In [11]:
print("Accuracy")
m = tf.keras.metrics.Accuracy()
m.update_state(y_test,
               np.array(predict))
m.result().numpy()

Accuracy


0.6424242

In [12]:
  print("Matrice de confusion")
  print("--------------------------------------------------------------------------------")

  tf.math.confusion_matrix(y_test, np.array(predict))

Matrice de confusion
--------------------------------------------------------------------------------


<tf.Tensor: shape=(10, 10), dtype=int32, numpy=
array([[20,  0,  3,  0,  0,  0,  0,  0,  0,  4],
       [ 0, 24,  1,  1,  0,  0,  0,  0,  0,  4],
       [ 2,  1, 19,  3,  1,  1,  0,  0,  2,  6],
       [ 0,  1,  1, 19,  1,  0,  0,  1,  1,  4],
       [ 0,  0,  1,  3, 19,  0,  1,  1,  7,  3],
       [ 3,  4,  3,  1,  0,  9,  0,  1,  2,  4],
       [ 1,  0,  0,  0,  0,  0, 33,  0,  0,  4],
       [ 0,  0,  2,  8,  0,  0,  0, 27,  2,  5],
       [ 1,  0,  1,  3,  1,  2,  0,  0, 26,  7],
       [ 2,  0,  2,  2,  0,  1,  0,  1,  1, 16]], dtype=int32)>